In [1]:
import numpy as np
import collections
import gym

import ptan
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class PGN(nn.Module):
    def __init__(self,input_size,n_actions):
        super(PGN,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,n_actions)
        )
        
    def forward(self,x):
        logits = self.net(x)
        return logits

In [3]:
class MeanBuffer():
    def __init__(self,capacity):
        self.capacity = capacity
        self.deque = collections.deque(maxlen=capacity)
        self.sum = 0.0
        
    def add(self,val):
        if len(self.deque)==self.capacity:
            self.sum -= self.deque[0]
        self.deque.append(val)
        self.sum+=val
    
    def mean(self):
        if not self.deque:
            return 0.0
        return self.sum/len(self.deque)

In [4]:
def getNewBatch(net,env,gamma,device='cpu',skip=1):
    while True:
        state = env.reset()
        probs =  F.softmax(net( torch.FloatTensor( state ).to(device) )).to('cpu').detach().numpy()
        action = np.random.choice( range(env.action_space.n), p=probs )
        states = [];actions = [];rewards = [];dones=[]
        done = False
        while not done:
            states.append(state)
            rewards.append(0)
            for _ in range(skip):
                state,reward,done,_ = env.step(action)
                rewards[-1]+=reward
                if done:
                    break
            actions.append(action)
            dones.append(done)

        total_rewards = sum(rewards)
        rewards.reverse()
        for i in range(1,len(rewards)):
            rewards[i] += gamma*rewards[i-1]
        rewards.reverse()

        for state,reward,action,done in zip(states,rewards,actions,dones):
            yield state,reward,action,done,total_rewards

In [4]:
TARGET_REWARD = 50
GAMMA = 0.99
LEARNING_RATE = 3e-4
BATCH_SIZE = 32
ENTROPY_BETA = 0.01
BELLMAN_STEPS = 5
BASELINE_STEPS = 50000

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make('LunarLander-v2')
net = PGN(env.observation_space.shape[0], env.action_space.n).to(device)

#agent = ptan.agent.PolicyAgent(net,device=device,preprocessor=ptan.agent.float32_preprocessor)
#exp_source = ptan.experience.ExperienceSourceFirstLast(env,agent,steps_count=BELLMAN_STEPS,gamma=GAMMA)

optimizer = optim.Adam(net.parameters(),lr=LEARNING_RATE)

total_rewards=  []
step_rewards = []
baseline_buff = MeanBuffer(BASELINE_STEPS)
step_idx= 0
done_episodes= 0

batch_states=[];batch_actions =[];batch_scales =[]
for state,reward,action,done,total_reward in getNewBatch(net,env,GAMMA,device):
    step_idx+=1
    baseline_buff.add(reward)
    baseline = baseline_buff.mean()
    batch_states.append(state)
    batch_actions.append(action)
    batch_scales.append(reward - baseline )

    if done:
        done_episodes += 1
        total_rewards.append(total_reward)
        mean_rewards = float(np.mean(total_rewards[-100:]))
        if done_episodes%50 == 0:
            print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (
                    step_idx, reward, mean_rewards, done_episodes))
        if mean_rewards > TARGET_REWARD:
            print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
            break

    if len(batch_states) < BATCH_SIZE:
        continue

    states_v = torch.FloatTensor(batch_states).to(device)
    actions_v = torch.LongTensor(batch_actions).to(device)
    scales_v = torch.FloatTensor(batch_scales).to(device)

    #apply gradient descent
    optimizer.zero_grad()

    #softmax output
    logit_v = net(states_v)
    log_prob_v = F.log_softmax(logit_v,dim=1)
    log_prob_action_v = scales_v * log_prob_v[range(BATCH_SIZE),actions_v]
    loss_policy_v = -log_prob_action_v.mean()

    prob_v = F.softmax(logit_v,dim=1)
    entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
    entropy_loss_v = -ENTROPY_BETA * entropy_v
    loss_v = loss_policy_v + entropy_loss_v

    loss_v.backward()
    optimizer.step()

    batch_states.clear()
    batch_actions.clear()
    batch_scales.clear()  

/home/aditya/.conda/envs/py3k/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
/home/aditya/.conda/envs/py3k/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


104: reward: -100.00, mean_100: -593.09, episodes: 1
165: reward: -100.00, mean_100: -572.45, episodes: 2
239: reward: -100.00, mean_100: -640.78, episodes: 3
488: reward: -100.00, mean_100: -1040.10, episodes: 4
574: reward: -100.00, mean_100: -868.06, episodes: 5
638: reward: -100.00, mean_100: -749.31, episodes: 6
725: reward: -100.00, mean_100: -790.00, episodes: 7
784: reward: -100.00, mean_100: -758.95, episodes: 8
846: reward: -100.00, mean_100: -689.13, episodes: 9
911: reward: -100.00, mean_100: -631.79, episodes: 10
1001: reward: -100.00, mean_100: -652.50, episodes: 11
1067: reward: -100.00, mean_100: -612.15, episodes: 12
1137: reward: -100.00, mean_100: -605.59, episodes: 13
1217: reward: -100.00, mean_100: -613.44, episodes: 14
1274: reward: -100.00, mean_100: -582.20, episodes: 15
1330: reward: -100.00, mean_100: -553.57, episodes: 16
1410: reward: -100.00, mean_100: -568.02, episodes: 17
1465: reward: -100.00, mean_100: -556.97, episodes: 18
1549: reward: -100.00, mean_

11082: reward: -100.00, mean_100: -175.29, episodes: 152
11169: reward: -100.00, mean_100: -175.76, episodes: 153
11254: reward: -100.00, mean_100: -175.88, episodes: 154
11308: reward: -100.00, mean_100: -169.36, episodes: 155
11365: reward: -100.00, mean_100: -169.18, episodes: 156
11450: reward: -100.00, mean_100: -169.91, episodes: 157
11532: reward: -100.00, mean_100: -169.77, episodes: 158
11614: reward: -100.00, mean_100: -170.13, episodes: 159
11689: reward: -100.00, mean_100: -170.54, episodes: 160
11777: reward: -100.00, mean_100: -166.91, episodes: 161
11860: reward: -100.00, mean_100: -167.34, episodes: 162
11912: reward: -100.00, mean_100: -167.25, episodes: 163
12000: reward: -100.00, mean_100: -163.27, episodes: 164
12070: reward: -100.00, mean_100: -163.41, episodes: 165
12146: reward: -100.00, mean_100: -164.30, episodes: 166
12200: reward: -100.00, mean_100: -163.91, episodes: 167
12282: reward: -100.00, mean_100: -164.15, episodes: 168
12342: reward: -100.00, mean_10

21438: reward: -100.00, mean_100: -132.42, episodes: 303
21510: reward: -100.00, mean_100: -131.26, episodes: 304
21592: reward: -100.00, mean_100: -132.94, episodes: 305
21676: reward: -100.00, mean_100: -132.65, episodes: 306
21733: reward: -100.00, mean_100: -132.12, episodes: 307
21817: reward: -100.00, mean_100: -131.96, episodes: 308
21879: reward: -100.00, mean_100: -132.17, episodes: 309
21931: reward: -100.00, mean_100: -132.26, episodes: 310
21985: reward: -100.00, mean_100: -131.99, episodes: 311
22047: reward: -100.00, mean_100: -131.58, episodes: 312
22103: reward: -100.00, mean_100: -131.23, episodes: 313
22187: reward: -100.00, mean_100: -130.93, episodes: 314
22264: reward: -100.00, mean_100: -130.64, episodes: 315
22341: reward: -100.00, mean_100: -130.95, episodes: 316
22410: reward: -100.00, mean_100: -131.17, episodes: 317
22462: reward: -100.00, mean_100: -130.57, episodes: 318
22548: reward: -100.00, mean_100: -130.74, episodes: 319
22614: reward: -100.00, mean_10

32049: reward: -100.00, mean_100: -134.24, episodes: 455
32136: reward: -100.00, mean_100: -134.74, episodes: 456
32206: reward: -100.00, mean_100: -134.92, episodes: 457
32292: reward: -100.00, mean_100: -134.42, episodes: 458
32371: reward: -100.00, mean_100: -134.71, episodes: 459
32442: reward: -100.00, mean_100: -134.72, episodes: 460
32531: reward: -100.00, mean_100: -133.53, episodes: 461
32615: reward: -100.00, mean_100: -133.12, episodes: 462
32701: reward: -100.00, mean_100: -133.55, episodes: 463
32773: reward: -100.00, mean_100: -134.01, episodes: 464
32828: reward: -100.00, mean_100: -134.28, episodes: 465
32892: reward: -100.00, mean_100: -134.68, episodes: 466
32948: reward: -100.00, mean_100: -134.62, episodes: 467
33014: reward: -100.00, mean_100: -134.80, episodes: 468
33077: reward: -100.00, mean_100: -134.80, episodes: 469
33166: reward: -100.00, mean_100: -135.84, episodes: 470
33234: reward: -100.00, mean_100: -135.99, episodes: 471
33313: reward: -100.00, mean_10

...

1237069: reward: -100.00, mean_100: -129.38, episodes: 17691
1237151: reward: -100.00, mean_100: -128.81, episodes: 17692
1237222: reward: -100.00, mean_100: -129.09, episodes: 17693
1237306: reward: -100.00, mean_100: -129.52, episodes: 17694
1237389: reward: -100.00, mean_100: -130.24, episodes: 17695
1237474: reward: -100.00, mean_100: -130.61, episodes: 17696
1237556: reward: -100.00, mean_100: -131.32, episodes: 17697
1237625: reward: -100.00, mean_100: -131.50, episodes: 17698
1237688: reward: -100.00, mean_100: -130.93, episodes: 17699
1237776: reward: -100.00, mean_100: -131.22, episodes: 17700
1237843: reward: -100.00, mean_100: -131.40, episodes: 17701
1237934: reward: -100.00, mean_100: -131.18, episodes: 17702
1238023: reward: -100.00, mean_100: -131.00, episodes: 17703
1238112: reward: -100.00, mean_100: -130.50, episodes: 17704
1238176: reward: -100.00, mean_100: -130.92, episodes: 17705
1238254: reward: -100.00, mean_100: -130.82, episodes: 17706
1238307: reward: -100.00

1246615: reward: -100.00, mean_100: -134.57, episodes: 17826
1246693: reward: -100.00, mean_100: -134.12, episodes: 17827
1246751: reward: -100.00, mean_100: -134.57, episodes: 17828
1246809: reward: -100.00, mean_100: -134.50, episodes: 17829
1246906: reward: -100.00, mean_100: -133.02, episodes: 17830
1246984: reward: -100.00, mean_100: -132.25, episodes: 17831
1247036: reward: -100.00, mean_100: -133.13, episodes: 17832
1247120: reward: -100.00, mean_100: -133.01, episodes: 17833
1247197: reward: -100.00, mean_100: -132.81, episodes: 17834
1247276: reward: -100.00, mean_100: -132.80, episodes: 17835
1247344: reward: -100.00, mean_100: -133.04, episodes: 17836
1247403: reward: -100.00, mean_100: -134.18, episodes: 17837
1247482: reward: -100.00, mean_100: -133.92, episodes: 17838
1247541: reward: -100.00, mean_100: -133.87, episodes: 17839
1247599: reward: -100.00, mean_100: -133.82, episodes: 17840
1247661: reward: -100.00, mean_100: -134.00, episodes: 17841
1247748: reward: -100.00

1256032: reward: -100.00, mean_100: -135.61, episodes: 17961
1256119: reward: -100.00, mean_100: -135.96, episodes: 17962
1256175: reward: -100.00, mean_100: -135.93, episodes: 17963
1256228: reward: -100.00, mean_100: -135.38, episodes: 17964
1256284: reward: -100.00, mean_100: -135.46, episodes: 17965
1256355: reward: -100.00, mean_100: -135.06, episodes: 17966
1256447: reward: -100.00, mean_100: -133.86, episodes: 17967
1256528: reward: -100.00, mean_100: -133.89, episodes: 17968
1256582: reward: -100.00, mean_100: -133.61, episodes: 17969
1256640: reward: -100.00, mean_100: -133.51, episodes: 17970
1256730: reward: -100.00, mean_100: -133.53, episodes: 17971
1256807: reward: -100.00, mean_100: -133.90, episodes: 17972
1256861: reward: -100.00, mean_100: -134.04, episodes: 17973
1256917: reward: -100.00, mean_100: -134.18, episodes: 17974
1256976: reward: -100.00, mean_100: -134.68, episodes: 17975
1257049: reward: -100.00, mean_100: -134.83, episodes: 17976
1257137: reward: -100.00

1265628: reward: -100.00, mean_100: -126.27, episodes: 18099
1265704: reward: -100.00, mean_100: -125.95, episodes: 18100
1265794: reward: -100.00, mean_100: -125.88, episodes: 18101
1265876: reward: -100.00, mean_100: -126.39, episodes: 18102
1265945: reward: -100.00, mean_100: -126.03, episodes: 18103
1266005: reward: -100.00, mean_100: -125.78, episodes: 18104
1266074: reward: -100.00, mean_100: -125.61, episodes: 18105
1266135: reward: -100.00, mean_100: -125.19, episodes: 18106
1266223: reward: -100.00, mean_100: -126.19, episodes: 18107
1266272: reward: -100.00, mean_100: -126.31, episodes: 18108
1266352: reward: -100.00, mean_100: -127.44, episodes: 18109
1266410: reward: -100.00, mean_100: -126.64, episodes: 18110
1266463: reward: -100.00, mean_100: -126.40, episodes: 18111
1266546: reward: -100.00, mean_100: -126.24, episodes: 18112
1266613: reward: -100.00, mean_100: -126.28, episodes: 18113
1266698: reward: -100.00, mean_100: -126.16, episodes: 18114
1266750: reward: -100.00

KeyboardInterrupt: 

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("LunarLander-v2")
net = PGN(env.observation_space.shape[0], env.action_space.n).to(device)
# The agent inputs the state into the neural network, and gets back logits.
# It puts these through a softmax to get probabilities, then samples an action.
agent = ptan.agent.PolicyAgent(net, preprocessor=ptan.agent.float32_preprocessor,
                               apply_softmax=True, device=device)
# The experience source interacts with the environment and returns (s,a,r,s') transitions
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=GAMMA, steps_count=BELLMAN_STEPS)

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

total_rewards = []
step_rewards = []
baseline_buf = MeanBuffer(BASELINE_STEPS)
step_idx = 0
done_episodes = 0

batch_states, batch_actions, batch_scales = [], [], []

# each iteration runs one action in the environment and returns a (s,a,r,s') transition
for step_idx, exp in enumerate(exp_source):
    baseline_buf.add(exp.reward)
    baseline = baseline_buf.mean()
    batch_states.append(exp.state)
    batch_actions.append(int(exp.action))
    batch_scales.append(exp.reward - baseline)

    # handle when an episode is completed
    episode_rewards = exp_source.pop_total_rewards()
    if episode_rewards:
        done_episodes += 1
        reward = episode_rewards[0]
        total_rewards.append(reward)
        mean_rewards = float(np.mean(total_rewards[-100:]))
        print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (
            step_idx, reward, mean_rewards, done_episodes))
        if mean_rewards > TARGET_REWARD:
            print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
            break

    if len(batch_states) < BATCH_SIZE:
        continue

    # copy training data to the GPU
    states_v = torch.FloatTensor(batch_states).to(device)
    batch_actions_t = torch.LongTensor(batch_actions).to(device)
    batch_scale_v = torch.FloatTensor(batch_scales).to(device)

    # apply gradient descent
    optimizer.zero_grad()
    logits_v = net(states_v)
    # apply the softmax and take the logarithm in one step, more precise
    log_prob_v = F.log_softmax(logits_v, dim=1)
    # scale the log probs according to (reward - baseline)
    log_prob_actions_v = batch_scale_v * log_prob_v[range(BATCH_SIZE), batch_actions_t]
    # take the mean cross-entropy across all batches
    loss_policy_v = -log_prob_actions_v.mean()

    # subtract the entropy bonus from the loss function
    prob_v = F.softmax(logits_v, dim=1)
    entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
    entropy_loss_v = -ENTROPY_BETA * entropy_v
    loss_v = loss_policy_v + entropy_loss_v

    loss_v.backward()
    optimizer.step()
    
    batch_states.clear()
    batch_actions.clear()
    batch_scales.clear()

/home/aditya/.conda/envs/py3k/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


101: reward: -316.80, mean_100: -316.80, episodes: 1
218: reward: -250.77, mean_100: -283.79, episodes: 2
282: reward: -60.48, mean_100: -209.35, episodes: 3
351: reward: -50.23, mean_100: -169.57, episodes: 4
433: reward: -129.47, mean_100: -161.55, episodes: 5
537: reward: -132.78, mean_100: -156.76, episodes: 6
615: reward: -99.09, mean_100: -148.52, episodes: 7
753: reward: -230.17, mean_100: -158.72, episodes: 8
828: reward: -202.63, mean_100: -163.60, episodes: 9
923: reward: -415.78, mean_100: -188.82, episodes: 10
1024: reward: -175.18, mean_100: -187.58, episodes: 11
1123: reward: -302.42, mean_100: -197.15, episodes: 12
1216: reward: -316.71, mean_100: -206.35, episodes: 13
1290: reward: -243.24, mean_100: -208.98, episodes: 14
1427: reward: -129.84, mean_100: -203.71, episodes: 15
1512: reward: -130.61, mean_100: -199.14, episodes: 16
1601: reward: -394.82, mean_100: -210.65, episodes: 17
1698: reward: -346.91, mean_100: -218.22, episodes: 18
1772: reward: -105.20, mean_100:

20198: reward: -187.43, mean_100: -227.74, episodes: 150
20348: reward: -125.85, mean_100: -227.90, episodes: 151
20498: reward: -140.87, mean_100: -228.55, episodes: 152
20666: reward: -144.49, mean_100: -229.22, episodes: 153
20832: reward: -111.59, mean_100: -228.33, episodes: 154
21015: reward: -220.69, mean_100: -228.05, episodes: 155
21238: reward: -172.31, mean_100: -225.27, episodes: 156
21361: reward: -278.54, mean_100: -223.19, episodes: 157
21539: reward: -143.14, mean_100: -222.06, episodes: 158
21693: reward: -187.33, mean_100: -220.05, episodes: 159
21825: reward: -329.94, mean_100: -218.10, episodes: 160
21977: reward: -122.92, mean_100: -217.65, episodes: 161
22139: reward: -264.72, mean_100: -218.09, episodes: 162
22319: reward:  -1.42, mean_100: -216.46, episodes: 163
22440: reward: -184.02, mean_100: -214.84, episodes: 164
22567: reward:   3.62, mean_100: -211.56, episodes: 165
22736: reward: -266.96, mean_100: -211.50, episodes: 166
22883: reward: -214.02, mean_100:

51580: reward: 166.91, mean_100: -76.15, episodes: 296
51816: reward: -59.12, mean_100: -75.06, episodes: 297
52284: reward: -57.79, mean_100: -72.41, episodes: 298
52504: reward: -16.37, mean_100: -71.13, episodes: 299
53012: reward:  -3.29, mean_100: -67.70, episodes: 300
53437: reward: 186.48, mean_100: -64.94, episodes: 301
53775: reward: -97.31, mean_100: -64.52, episodes: 302
53979: reward: -120.43, mean_100: -65.57, episodes: 303
54260: reward: -79.65, mean_100: -63.79, episodes: 304
54493: reward:  24.44, mean_100: -62.71, episodes: 305
54836: reward: -349.26, mean_100: -66.28, episodes: 306
55837: reward:  84.67, mean_100: -64.96, episodes: 307
56097: reward: -42.26, mean_100: -64.89, episodes: 308
56403: reward: -98.20, mean_100: -65.51, episodes: 309
56771: reward: -41.17, mean_100: -63.78, episodes: 310
56939: reward:  32.17, mean_100: -62.37, episodes: 311
57940: reward:  -9.64, mean_100: -62.51, episodes: 312
58890: reward: -189.67, mean_100: -61.96, episodes: 313
59476: 

114814: reward: -66.06, mean_100: -31.33, episodes: 444
115815: reward:  60.40, mean_100: -30.37, episodes: 445
116067: reward: -18.77, mean_100: -29.74, episodes: 446
116455: reward: -75.35, mean_100: -29.93, episodes: 447
116609: reward:  35.09, mean_100: -29.96, episodes: 448
116819: reward: -24.77, mean_100: -30.08, episodes: 449
117218: reward: -48.00, mean_100: -30.14, episodes: 450
117490: reward:  27.79, mean_100: -29.88, episodes: 451
118491: reward:  85.70, mean_100: -30.08, episodes: 452
118924: reward: -115.97, mean_100: -31.13, episodes: 453
119073: reward: -151.12, mean_100: -31.48, episodes: 454
120074: reward: 141.44, mean_100: -28.68, episodes: 455
120252: reward:  37.01, mean_100: -28.34, episodes: 456
120520: reward:  43.51, mean_100: -27.65, episodes: 457
120931: reward: -37.14, mean_100: -27.94, episodes: 458
121272: reward:   2.43, mean_100: -27.70, episodes: 459
121436: reward:  10.41, mean_100: -26.80, episodes: 460
121612: reward:  68.48, mean_100: -26.08, epis

171849: reward: -40.94, mean_100: -36.67, episodes: 590
172160: reward: -203.00, mean_100: -38.00, episodes: 591
172445: reward: -69.28, mean_100: -37.16, episodes: 592
173050: reward: 185.14, mean_100: -35.98, episodes: 593
174051: reward:  21.14, mean_100: -37.33, episodes: 594
174709: reward: 123.52, mean_100: -36.60, episodes: 595
174911: reward:  36.06, mean_100: -36.24, episodes: 596
175814: reward: 105.48, mean_100: -36.61, episodes: 597
176815: reward: 199.51, mean_100: -36.17, episodes: 598
177046: reward: -36.25, mean_100: -37.34, episodes: 599
177225: reward: -54.49, mean_100: -36.38, episodes: 600
177320: reward:  20.61, mean_100: -34.48, episodes: 601
178321: reward: 105.02, mean_100: -31.56, episodes: 602
178560: reward:  -6.66, mean_100: -33.37, episodes: 603
178759: reward:  10.88, mean_100: -33.02, episodes: 604
179170: reward: 152.00, mean_100: -30.08, episodes: 605
179406: reward:  15.29, mean_100: -29.57, episodes: 606
179806: reward: -42.63, mean_100: -28.24, episo